In [1]:
from scipy import sparse
import pandas as pd
import xgboost

In [2]:
X_tfidf_sample = sparse.load_npz('X_consolidated_without_numbers.npz')

In [3]:
print("shape of TF_IDF:")
X_tfidf_sample.shape

shape of TF_IDF:


(98728, 64055)

In [4]:
X_train = X_tfidf_sample[:84916,:]
X_test = X_tfidf_sample[84916:,:]
print(X_train.shape, X_test.shape )

(84916, 64055) (13812, 64055)


# XGBoost first search

In [5]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [6]:
Y= pd.read_csv('Y_train_CVw08PX.csv')
type(Y)

pandas.core.frame.DataFrame

In [7]:
y_train = Y['prdtypecode']    # label

In [55]:
model = xgboost.XGBClassifier(n_estimators=100, max_depth=8, learning_rate=0.1, subsample=0.5, verbosity = 2)

train_model = model.fit(X_train, y_train)

[22:23:11] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 54 extra nodes, 0 pruned nodes, max_depth=8
[22:23:12] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 54 extra nodes, 0 pruned nodes, max_depth=8
[22:23:12] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 52 extra nodes, 0 pruned nodes, max_depth=8
[22:23:12] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 46 extra nodes, 0 pruned nodes, max_depth=8
[22:23:12] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 20 extra nodes, 0 pruned nodes, max_depth=8
[22:23:12] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 26 extra nodes, 0 pruned nodes, max_depth=8
[22:23:12] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 20 extra nodes, 0 pruned nodes, max_depth=8
[22:23:13] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 56 extra nodes, 0 pruned nodes, max_depth=8
[22:23:13] INFO: /workspace/src/tree/upd

In [56]:
XGBoost_predictions = train_model.predict(X_test)

In [57]:
XGBoost_predictions

array([2280, 1160, 2583, ..., 2583, 2582, 2060])

In [70]:
my_index  =list(range(84916, 98728))

In [76]:
XGBoost_predictions
my_index  =list(range(84916, 98728))
df = pd.DataFrame(data = XGBoost_predictions, columns = ["prdtypecode"])
df['my_index'] = my_index
df = df.set_index('my_index')
df

,prdtypecode
my_index,
84916,2280
84917,1160
84918,2583
84919,2583
84920,2522
...,...
98723,1560
98724,2403
98725,2583


In [79]:
df.to_csv('rakuten_submission.csv', index = True, header = 'True')

TypeError: to_csv() got an unexpected keyword argument 'delimiter'

In [11]:
pip install sklearn

Note: you may need to restart the kernel to use updated packages.


In [19]:
# Option 2:
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV

xgb_model = XGBClassifier()

random_grid = {
 "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ] }

print(random_grid)

{'learning_rate': [0.05, 0.1, 0.15, 0.2, 0.25, 0.3], 'max_depth': [3, 4, 5, 6, 8, 10, 12, 15], 'min_child_weight': [1, 3, 5, 7], 'gamma': [0.0, 0.1, 0.2, 0.3, 0.4], 'colsample_bytree': [0.3, 0.4, 0.5, 0.7]}


In [20]:
xgb_random = RandomizedSearchCV(estimator = xgb_model, param_distributions = random_grid,
                               n_iter = 20, cv = 3, verbose=2, random_state=42 ,n_jobs = -1, scoring = 'f1_weighted')

# Fit the random search model
xgb_random.fit(X_train, y_train.values.ravel())

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  7.9min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 47.5min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 47.5min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n...
                                           verbosity=None),
                   iid='deprecated', n_iter=20, n_jobs=-1,
                   param_distributions={'colsample_bytree': [

In [27]:
xgb_random.cv_results_

{'mean_fit_time': array([ 461.05060673,  866.19945272,  830.02566528,  403.06949306,
         340.87095165,  281.15718738, 1730.45293045, 1268.42478585,
         287.85730553,  998.91413546,  416.70512064,  653.6400152 ,
         271.34131265,  344.92450507,  778.19968915,  969.74751083,
         352.92330027,  457.76393906,  784.23214436,  173.26423256]),
 'std_fit_time': array([ 1.85378534,  0.90574573,  4.45963002,  0.60349841,  0.54740091,
         0.80258162,  2.08453361,  4.01180168,  0.1747095 ,  2.68580448,
         1.57076572,  1.66273728,  0.44134516,  0.62610741,  1.7322298 ,
        19.3642394 ,  0.97240886,  1.96524934, 38.9305965 ,  4.29275378]),
 'mean_score_time': array([11.64281996, 20.74397588, 38.59974639, 15.91487137, 13.59556341,
        11.31853422, 32.17051347, 27.75019534, 12.1232907 , 29.17344626,
        20.61268107, 24.51094834, 13.80134964, 13.77260574, 22.50195519,
        21.38306808,  9.12785085, 11.230443  , 20.57621606,  7.69909994]),
 'std_score_time':

In [21]:
xgb_random.best_params_

{'min_child_weight': 1,
 'max_depth': 15,
 'learning_rate': 0.2,
 'gamma': 0.4,
 'colsample_bytree': 0.7}

In [23]:
model = xgboost.XGBClassifier(n_estimators=100, max_depth=15, learning_rate=0.2,gamma = 0.4, colsample_bytree = 0.7, verbosity = 1)

train_model = model.fit(X_train, y_train)

In [24]:
XGBoost_predictions = train_model.predict(X_test)
my_index  =list(range(84916, 98728))
df = pd.DataFrame(data = XGBoost_predictions, columns = ["prdtypecode"])
df['my_index'] = my_index
df = df.set_index('my_index')
df

,prdtypecode
my_index,
84916,1280
84917,1160
84918,2583
84919,2583
84920,2522
...,...
98723,1280
98724,2403
98725,2583


In [25]:
df.to_csv('rakuten_submission_xgb_rg.csv', index = True, header = 'True')

# XGBoost more extensive random search

In [31]:
# Option 2:
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV

xgb_model = XGBClassifier()

random_grid = {
 "n_estimators"     : [5, 20, 50, 100, 200, 500],
 "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ] }

print(random_grid)

{'n_estimators': [5, 20, 50, 100, 200, 500], 'learning_rate': [0.05, 0.1, 0.15, 0.2, 0.25, 0.3], 'max_depth': [3, 4, 5, 6, 8, 10, 12, 15], 'min_child_weight': [1, 3, 5, 7], 'gamma': [0.0, 0.1, 0.2, 0.3, 0.4], 'colsample_bytree': [0.3, 0.4, 0.5, 0.7]}


In [32]:
xgb_random = RandomizedSearchCV(estimator = xgb_model, param_distributions = random_grid,
                               n_iter = 50, cv = 3, verbose=2, random_state=42 ,n_jobs = -1, scoring = 'f1_weighted')

# Fit the random search model
xgb_random.fit(X_train, y_train.values.ravel())

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 161.1min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n...
                   iid='deprecated', n_iter=50, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.3, 0.4, 0.5,
                                             

In [33]:
xgb_random.best_params_

{'n_estimators': 500,
 'min_child_weight': 1,
 'max_depth': 6,
 'learning_rate': 0.3,
 'gamma': 0.3,
 'colsample_bytree': 0.4}

In [34]:
xgb_random.cv_results_

{'mean_fit_time': array([ 459.35227791,  160.38811016,   93.29134973, 2755.78754568,
          22.85464629, 1733.63477802,  240.74151707,  154.54018625,
         555.85048413,  381.73104541,  406.07706062, 1398.80157797,
        1502.88984696,  508.57880743,   19.70843927, 3176.41634512,
          57.74137068,  558.68544443,  112.30424706, 1359.85271343,
          22.38324873,   49.81304351, 1813.4960142 , 6909.02980113,
          19.06178292,  161.34743031,   60.00861669, 6362.61886954,
        1369.67934227,  765.03993964,  447.81393711, 2598.66887093,
          46.55234337,  821.65419404,   87.40561191,  297.31283847,
          18.47581013,  950.81340098,  155.72748105,   13.42324527,
          36.3080047 ,  674.15377601,   72.85052554,  412.7500244 ,
         199.43576447, 1854.40846602, 1021.51244768,  353.99443499,
          24.42295051,  206.34722368]),
 'std_fit_time': array([1.50793830e+00, 8.36980711e-01, 1.83834928e-01, 1.09810195e+01,
        1.54200118e-01, 6.36020683e+00,

'n_estimators': 500,
 'min_child_weight': 1,
 'max_depth': 6,
 'learning_rate': 0.3,
 'gamma': 0.3,
 'colsample_bytree': 0.4

In [ ]:
# Train the model with the best parameters
model = xgboost.XGBClassifier(n_estimators=1000, max_depth=6, learning_rate=0.3 ,gamma = 0.3, colsample_bytree =0.4 , min_child_weight = 1, verbosity = 1)

train_model = model.fit(X_train, y_train)

In [39]:
XGBoost_predictions = train_model.predict(X_test)
my_index  =list(range(84916, 98728))
df = pd.DataFrame(data = XGBoost_predictions, columns = ["prdtypecode"])
df['my_index'] = my_index
df = df.set_index('my_index')
df

,prdtypecode
my_index,
84916,1280
84917,1160
84918,2583
84919,2583
84920,2522
...,...
98723,1560
98724,2403
98725,2583


In [40]:
# Export results to pdf
df.to_csv('rakuten_submission_xgb_rg3.csv', index = True, header = 'True')

# Decision Tree Random Search

In [ ]:
# Decision treee classifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV

clf_dt = DecisionTreeClassifier()

random_grid = {
 "criterion"     : ['gini', 'entropy'],
 "max_depth"     : [4, 6, 8, 12, 20, 40,50] ,
 "splitter"      : ["best", "random"]}

print(random_grid)

In [ ]:
dt_random = RandomizedSearchCV(estimator = clf_dt, param_distributions = random_grid,
                               n_iter = 30, cv = 3, verbose=2, random_state=42 ,n_jobs = -1, scoring = 'f1_weighted')

# Fit the random search model
dt_random.fit(X_train, y_train.values.ravel())

In [ ]:
dt_random.best_params_

In [ ]:
dt_random.cv_results_

In [ ]:
# Train the model with the best parameters
dt_model = DecisionTreeClassifier(criterion= 'entropy' ,max_depth = 50,splitter = 'random')

dt_train_model = dt_model.fit(X_train, y_train)

In [ ]:
DT_predictions = dt_train_model.predict(X_test)
my_index  =list(range(84916, 98728))
df = pd.DataFrame(data = DT_predictions, columns = ["prdtypecode"])
df['my_index'] = my_index
df = df.set_index('my_index')
df

In [ ]:
# Export results to pdf
df.to_csv('rakuten_submission_dt_rg.csv', index = True, header = 'True')

# Default Decision Tree

In [2]:
# The fact that the there is a limit on the max_depth is eliminates the performance of the DecisionTree,
# I will try a decision tree with no max_depth

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [ ]:
my_index = list(range(84916, 98728))
df_tree = pd.DataFrame(data = y_pred, columns = ["prdtypecode"])
df_tree['my_index'] = my_index
df_tree = df_tree.set_index('my_index')
df_tree
df_tree.to_csv('rakuten_submission_dt_latest.csv', index = True, header = 'True')